In [2]:
# Load model directly
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1", trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-R1:
- configuration_deepseek.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-R1:
- modeling_deepseek.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ValueError: Unknown quantization type, got fp8 - supported types are: ['awq', 'bitsandbytes_4bit', 'bitsandbytes_8bit', 'gptq', 'aqlm', 'quanto', 'eetq', 'hqq', 'compressed-tensors', 'fbgemm_fp8', 'torchao']

# Distilled Model

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

/Users/hfsg/miniconda3/envs/huggingface/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

In [3]:
type(model)

transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM

In [11]:
from transformers import Qwen2Tokenizer, Qwen2ForCausalLM

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
input_text = "Help me write a graph data structure implementation in python"  # Replace with your prompt

inputs = tokenizer(input_text, return_tensors="pt")
inputs

{'input_ids': tensor([[151646,  12689,    752,   3270,    264,   4771,    821,   5944,   8129,
            304,  10135]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [20]:
import torch

with torch.no_grad():
    # output = model(**inputs)
    outputs = model.generate(**inputs, max_length=1000, do_sample=True, temperature=0.7)

# generated_text = tokenizer.decode(output.logits[0].argmax(dim=-1))
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [21]:
print(generated_text)

<｜begin▁of▁sentence｜>Help me write a graph data structure implementation in python.
Okay, I need to write a graph data structure in Python, and I need to make sure it's efficient and easy to use. I remember that a graph consists of nodes (also called vertices) and edges, which connect these nodes. Each edge can be either directed or undirected. For this project, I think I should focus on undirected graphs because I'm more comfortable with that. 

Wait, but how do I represent an undirected graph in Python? I've heard of adjacency lists, where each node has a list of its connected nodes. That sounds manageable. So, maybe I can create a class for the graph that has a dictionary where each key is a node, and the value is the list of connected nodes. 

But before I jump into coding, I should think about the structure of the graph. Should I include a method to add nodes and edges? I think yes, because users will need to create and modify the graph as they go. Also, I should consider the poss

# Distilled Dataset

In [22]:
from datasets import load_dataset

ds = load_dataset("open-thoughts/OpenThoughts-114k")
print(ds.shape)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Generating train split: 100%|██████████| 113957/113957 [00:02<00:00, 41767.12 examples/s]


In [24]:
ds

DatasetDict({
    train: Dataset({
        features: ['system', 'conversations'],
        num_rows: 113957
    })
})

In [28]:
ds["train"][0].keys()

dict_keys(['system', 'conversations'])

In [27]:
print(ds["train"][0]["system"])

"Your role as an assistant involves thoroughly exploring questions through a systematic long thinking process before providing the final precise and accurate solutions. This requires engaging in a comprehensive cycle of analysis, summarizing, exploration, reassessment, reflection, backtracing, and iteration to develop well-considered thinking process. Please structure your response into two main sections: Thought and Solution. In the Thought section, detail your reasoning process using the specified format: <|begin_of_thought|> {thought with steps separated with '\\n\\n'} <|end_of_thought|> Each step should include detailed considerations such as analisying questions, summarizing relevant findings, brainstorming new ideas, verifying the accuracy of the current steps, refining any errors, and revisiting previous steps. In the Solution section, based on various attempts, explorations, and reflections from the Thought section, systematically present the final solution that you deem correc

In [34]:
ds["train"][0]["conversations"][0]["from"]

'user'

In [37]:
print(ds["train"][0]["conversations"][0]["value"])

Generate an executable Python function generated from the given prompt. The function should take stdin as input and print the output. Simply call the function after the definition.The Chef likes to stay in touch with his staff. So, the Chef, the head server, and the sous-chef all carry two-way transceivers so they can stay in constant contact. Of course, these transceivers have a limited range so if two are too far apart, they cannot communicate directly.


The Chef invested in top-of-the-line transceivers which have a few advanced features. One is that even if two people cannot talk directly because they are out of range, if there is another transceiver that is close enough to both, then the two transceivers can still communicate with each other using the third transceiver as an intermediate device.


There has been a minor emergency in the Chef's restaurant
and he needs to communicate with both the head server and the sous-chef right away. Help the Chef determine if it is possible fo

In [38]:
ds["train"][0]["conversations"][1]["from"]

'assistant'

In [40]:
print(ds["train"][0]["conversations"][1]["value"])

<|begin_of_thought|>

Okay, let's see. I need to solve this problem where three people (Chef, head server, sous-chef) have transceivers that can communicate directly if within R meters. If not, but there's a third person acting as a bridge, then they can still communicate. The goal is to check if all three can communicate with each other, possibly through the third.

Hmm. So for each test case, given R and three points, determine if the three form a connected graph where each pair is either directly connected (distance <= R) or connected via the third.

Wait, but how exactly? Let's think. The communication can go through one intermediary. So all three must be in a chain where each consecutive pair is within R, or perhaps any two are connected via a path through the third.

Wait, the problem says: if two can't talk directly but there's a third that is close enough to both, then they can communicate through that third. So the communication graph must be connected. But the three people ca

In [41]:
def main():
    import sys
    input = sys.stdin.read().split()
    ptr = 0
    T = int(input[ptr])
    ptr += 1
    for _ in range(T):
        R = int(input[ptr])
        ptr += 1
        A = list(map(int, input[ptr:ptr+2]))
        ptr += 2
        B = list(map(int, input[ptr:ptr+2]))
        ptr += 2
        C = list(map(int, input[ptr:ptr+2]))
        ptr += 2
        count = 0
        
        # Check Chef to Head Server
        dx = A[0] - B[0]
        dy = A[1] - B[1]
        if dx * dx + dy * dy <= R * R:
            count += 1
        
        # Check Head Server to Sous-Chef
        dx = B[0] - C[0]
        dy = B[1] - C[1]
        if dx * dx + dy * dy <= R * R:
            count += 1
        
        # Check Sous-Chef to Chef
        dx = C[0] - A[0]
        dy = C[1] - A[1]
        if dx * dx + dy * dy <= R * R:
            count += 1
        
        print("yes" if count >= 2 else "no")

main()

KeyboardInterrupt: 